## Init

In [1]:
import yaml
import os
import sys
from pathlib import Path
to_utils_path = Path(os.getcwd()).parent
sys.path.append(str(to_utils_path))
to_core_path = Path(os.getcwd()).parent.parent.joinpath("core")
sys.path.append(str(to_core_path))

%matplotlib inline
#%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

from yaml_creator import *

Autosaving every 180 seconds


c:\Users\mauls\miniconda3\envs\neo_cebra\lib\site-packages\numba\core\decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


## Spreadsheet Settings

In [4]:
# Re-structuring the data
root_dir = "D:\\Experiments"
experimentalist = "Nathalie"
paradigm = "OpenFieldDynamics_2"
animal_root_dir = os.path.join(root_dir, experimentalist, paradigm)
path = Path(animal_root_dir)
e_animal_comment = {
    "DON-007021": "all sessions usefull (caution: animal explored little in sessions FS12, FS16, FS18)",
    "DON-007050": "all sessions usefull (caution: imaging FOV was out of focus during session NS3)",
    "DON-007369": "only use sessions FS7 till FS12 to look at the effect of experience on a familiar map",
    "DON-007416": "all sessions usefull",
    "DON-007417": "all sessions usefull (caution: session NS8 is missing due to recording error)",
    "DON-007513": "all sessions usefull",
    "DON-018116": "all sessions usefull",
    "DON-018154": "all sessions usefull (caution: focus of miniscope shifted during session NS3)",
    "DON-020762": "use only use sessions FS1 till FS9 to look at spatial map formation in naïve animals as the mouse didn't learn",
}

control_animal_comment = {
    "DON-014515": "all sessions usefull",
    "DON-015128": "all sessions usefull",
    "DON-016562": "all sessions usefull (caution: animal explored little in session FS2 and imaging FOV was out of focus in FS14, NS4, NS5)",
    "DON-016584": "all sessions usefull (caution: animal explored little in session FS16)",
    "DON-018115": "all sessions usefull",
    "DON-020765": "all sessions usefull",
}

In [8]:
from restructure import forbidden_names
from pathlib import Path
from datetime import datetime
from Helper import *
from tqdm import tqdm

folder_name_strcture = {
    "animal": r"^DON-\d{6,8}$",
    "date": r"^\d{8}$",
}

animal_dict = {
    "animal_id": None,
    "dob": 19990101,
    "expt_pipeline": "OpenFieldDynamics",
    "condition": None,
    "comment": None,
}

neural_metadata = {
    "method": "1P",
    "area": "CA3",
    "setup": "inscopix",
    "preprocessing": "opexebo",
    "processing": "cabincorr",
    "fps": 20,
}
behavior_metadata = {
    "setup": "openfield",
    "preprocessing": "cam",
    "binning_size": 0.05,  # meters #default is 0.05 for 2D and 0.01 for 1D
    "stimulus_type": None,
    "environment_dimensions": [0.80, 0.80],  # meters
    "pixel_per_meter": None,  # 1.35625 m per pixel
    # for NS sessions, the pixel_per_meter is 13562.
    # for FS sessions, the pixel_per_meter is 1547.8
}

task_dict = {
    "neural_metadata": neural_metadata,
    "behavior_metadata": behavior_metadata,
}

date_dict = {"tasks_infos": {"task_name": task_dict}}

animal_folders = search_filedir(
    path=path,
    include_regex=folder_name_strcture["animal"],
    type="dir",
)
for animal_folder in tqdm(animal_folders, desc="animal folders"):
    animal_id = animal_folder.name
    animal_dir = path.joinpath(animal_id)
    if animal_id in e_animal_comment.keys():
        animal_dict["comment"] = e_animal_comment[animal_id]
        animal_dict["condition"] = "experimental"
    elif animal_id in control_animal_comment.keys():
        animal_dict["comment"] = control_animal_comment[animal_id]
        animal_dict["condition"] = "control"
    else:
        animal_dict["comment"] = "unknown animal"

    animal_dict["animal_id"] = animal_id

    date_folders = search_filedir(
        path=animal_dir,
        include_regex=folder_name_strcture["date"],
        type="dir",
    )
    for date_folder in date_folders:
        date = date_folder.name
        date_dir = animal_dir.joinpath(date)
        date_dict = {
            "date": date,
            "tasks_infos": {},
        }

        task_folders = search_filedir(
            path=date_dir,
            exclude_regex=forbidden_names,
            type="dir",
        )

        for task_folder in task_folders:
            task_name = task_folder.name
            task_dir = date_dir.joinpath(task_name)

            if "NS" in task_name:
                task_dict["behavior_metadata"]["pixel_per_meter"] = 1356.25
                task_dict["behavior_metadata"]["stimulus_type"] = "novel"
            elif "FS" in task_name:
                task_dict["behavior_metadata"]["pixel_per_meter"] = 1547.8
                task_dict["behavior_metadata"]["stimulus_type"] = "familiar"
            else:
                print(f"Unknown task name: {task_name}.")
                continue

            # Add the task metadata to the date_dict
            date_dict["tasks_infos"][task_name] = task_dict

        # save the date_dict to a yaml file
        with open(date_dir.joinpath(f"{date}.yaml"), "w") as f:
            yaml.dump(date_dict, f, default_flow_style=False, sort_keys=False)

    # save the animal_dict to a yaml file
    with open(animal_dir.joinpath(f"{animal_id}.yaml"), "w") as f:
        yaml.dump(animal_dict, f, default_flow_style=False, sort_keys=False)

animal folders:  47%|████▋     | 7/15 [00:00<00:00, 68.62it/s]

Unknown task name: models.


animal folders: 100%|██████████| 15/15 [00:00<00:00, 69.76it/s]
